# Crée des images à partir des images existantes

In [1]:
# !pip install import-ipynb

In [2]:
import import_ipynb

In [3]:
%run iteration_0_imports.py

In [4]:
%run iteration_0_parameters.py

In [5]:
import iteration_0_utils as utils

importing Jupyter notebook from iteration_0_utils.ipynb


In [6]:
%whos

Variable                          Type        Data/Info
-------------------------------------------------------
HEALTHY                           str         H
IMG_IN_COLOR                      int         1
IMG_SIZE                          int         64
Image                             module      <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
NBR_SAMPLE                        int         20000
PATH_DATA                         str         ../data
PATH_DATA_EDA                     str         ../data/eda
PATH_DATA_LIST                    str         ../data/list
PATH_DATA_SAMPLE                  str         ../data/sample
PATH_MODEL                        str         ../model
PATH_ROOT                         str         ..
PATH_SOURCE_IMG                   str         ../cell_images
RandomForestClassifier            ABCMeta     <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
SICK                              str         S
classification_report             function

In [7]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [14]:
df_path_img_sick.head()

,path_img
0,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_162.png
1,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_163.png
2,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_164.png
3,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_165.png
4,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_166.png


In [15]:
PATH_DATA_SAMPLE = PATH_DATA + "/" + "generated"

In [16]:
NBR_SAMPLE = 10